In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator = "\n",
    chunk_size = 300,
    chunk_overlap = 50,
)

loader = UnstructuredFileLoader('./files/george_orwell.csv')

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# vectorstore = Chroma.from_documents(docs,cached_embeddings)

vectorstore = FAISS.from_documents(docs,cached_embeddings)

# chain = RetrievalQA.from_chain_type(
#     llm = llm,
#     chain_type = "stuff",
#     retriever = vectorstore.as_retriever(),
# )

# chain = RetrievalQA.from_chain_type(
#     llm = llm,
#     chain_type = "refine",
#     retriever = vectorstore.as_retriever(),
# )

# chain = RetrievalQA.from_chain_type(
#     llm = llm,
#     chain_type = "map_reduce",
#     retriever = vectorstore.as_retriever(),
# )

chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "map_rerank",
    retriever = vectorstore.as_retriever(),
)

# vectorstore.similarity_search("where is winston house?")

chain.run("Describe Victory Mansions?")

'Victory Mansions is a rundown, dilapidated apartment complex located in London, depicted in George Orwell\'s novel "1984." The building has rotting nineteenth-century houses with sides shored up with timber, windows patched with cardboard, and roofs made of corrugated iron. Surrounding the area are bombed sites, sagging garden walls, and sordid colonies of wooden dwellings. Inside, there are telescreens constantly broadcasting Party propaganda, creating a cold and austere atmosphere. The living conditions are grim, with shortages of basic necessities, lack of privacy due to constant surveillance, and patrols by the Thought Police.'